This is v2 of the control law propagator.

The major update to the CLP is that there is an extra loop for propagation; Inside a single bucket, use a fixed step size and hold u constant while we propagate q and p.  Then, using those values for p and q, propagate u.

There will be three main modules:

- 1) Propagator

- 2) sliding window

- 3) window filter

In [6]:
import numpy as np
import scipy as sp
import ode

In [7]:
# Inputs:
q_0 = np.array([0])
p_0 = np.array([0])
u_0 = np.array([0])
qpu_vec = np.hstack([q_0, p_0, u_0])

t_0 = 0
T =  2
K=1

n_s = 10
integrateTol = 10**-3
integrateMaxIter = 40
state_dim = 1
Gamma = 1

In [8]:
def H_T_p(q,p,u):
    # for q-dot
    return np.ones(np.shape(q))*0

def H_T_q(q,p,u):
    # for p-dot
    return np.ones(np.shape(p))*0
    
def Q_u(q,p,u):
    # for u-dot
    return np.ones(np.shape(u))*0
    

In [9]:
def rhs(t, qpu_vec, **kwargs):
    # TODO: make sure that the functions that this calls are available
    state_dim = kwargs['state_dim']
    Gamma = kwargs['Gamma']
    q = qpu_vec[:state_dim]
    p = qpu_vec[state_dim:2*state_dim]
    u = qpu_vec[2*state_dim:]
    q_dot =  H_T_p(q,p,u)
    p_dot = -1*H_T_q(q,p,u)
    u_dot = -Gamma*Q_u(q,p,u)
    return np.hstack([q_dot, p_dot, u_dot])

In [10]:

steps = range(t_start,t_end, float(t_end-t_start)/float(n_s))

TypeError: range() integer step argument expected, got float.

In [11]:
import numpy as np

In [5]:
def propagate_dynamics(t_0, T, K, qpu_vec, integrateTol, integrateMaxIter, state_dim, Gamma, n_s):
    '''
    n_s is number of steps
    '''
    qs=[]
    ps=[]
    us=[]

    ts = range(t_0,T+1,(T-t_0)/(2*K))  # go until T+1 because last value will be used as starting point for next window

    for i in range(len(ts)-1):
        # starting value of u for a single bucket
        t_start, t_end = ts[i], ts[i+1]
        qp_vecs = propagate_q_p(u_0, q_0, p_0, t_start, t_end, n_s)  # assume "u" constant, and propagate q and p
        u_vecs = propagate_u(u_0, qp_vecs, t_start, t_end, n_s)      # pass in the resulting q and p values to be used for propagating the "u"
        
        qpu_vec_i = np.hstack([qp_vecs, u_vecs])
        qpu_vec = qpu_vec_i[-1] # only need the last value
        if i == len(ts)-2:
            pass
            # no need to append since weight = 0 for last value.  But qpu_vec still needs to be updated.
        else:
            qs.append(qpu_vec[:state_dim])
            ps.append(qpu_vec[state_dim:2*state_dim])
            us.append(qpu_vec[2*state_dim:])
    return qpu_vec, qs, ps, us  # return values for one entire window

    
def propagate_q_p(u_0, q_0, p_0, t_start, t_end, n_s):
    '''
    Propagate q and p to end of bucket using rk23
    '''
    qp_vecs = []
    qp_vec = [q_0, p_0]
    steps = np.linspace(t_start,t_end, n_s)
    for i in range(n_s):
        n_start, n_end = steps[i], steps[i+1]
        # TODO: pass in ONLY the q dot and p dot dynamic equations for the rhs
        qp_vec, t, failFlag, iter_i = ode.ode_rk23(rhs, n_start, n_end, qp_vec, integrateTol, integrateMaxIter, state_dim=state_dim, Gamma = Gamma)
        qp_vecs.append(qp_vec[-1])
    return qp_vecs
    
    
def propagate_u(u_0, q_0, p_0, n_s): 
    '''
    '''
    
    

In [6]:
qpu_vec, qs, ps, us = propagate_dynamics(t_0, T, K, qpu_vec, integrateTol, integrateMaxIter, state_dim, Gamma)

In [7]:
qpu_vec

array([0., 0., 0.])

### Triangle window filter

In [8]:
def get_weights(K):
    weights_0 = [float(i)/K for i in range(1,K+1)]  
    weights_1 = [2-(float(i)/K) for i in range(K+1,(2*K)+1)]
    # sanity check 
    assert len(weights_0)==len(weights_1)
    weights = weights_0+weights_1
    weights_total = sum(weights[:-1])
    return weights, weights_total

In [9]:
def apply_filter(vec, weights, weights_total):
    vec_weighted = [val*w for val,w in zip(vec, weights[:-1])]
    vec_current = np.sum(vec_weighted,0)
    vec_normalized = vec_current/float(weights_total)
    return vec_normalized

weights, weights_total = get_weights(K)
q_bar = apply_filter(qs,weights, weights_total)
p_bar = apply_filter(ps,weights, weights_total)
u_bar = apply_filter(us,weights, weights_total)

In [10]:
# outputs:

print q_bar
print p_bar
print u_bar

print qs
print ps
print us

[0.]
[0.]
[0.]
[array([0.])]
[array([0.])]
[array([0.])]


### Sliding window (outer loop)

In [11]:
# additional inputs 
t_terminal = 100

In [12]:
def sliding_window(t_0, T, K, q_0, p_0, u_0, state_dim, Gamma, t_terminal):
    q_bars = []
    p_bars = []
    u_bars = []
    weights, weights_total = get_weights(K)
    t=t_0 # wall clock time
    qpu_vec = np.hstack([q_0, p_0, u_0])
    while t<t_terminal:
        
        qpu_vec, qs, ps, us = propagate_dynamics(t_0, T, K, qpu_vec, integrateTol, integrateMaxIter, state_dim, Gamma)
        # qs, ps, and us will go to Mean Field somehow

        q_bar = apply_filter(qs,weights, weights_total)
        p_bar = apply_filter(ps,weights, weights_total)
        u_bar = apply_filter(us,weights, weights_total)
        
        t+=1
        
        q_bars.append(q_bar)
        p_bars.append(p_bar)
        u_bars.append(u_bar)

    return q_bars, p_bars, u_bars

In [13]:
q_bars, p_bars, u_bars = sliding_window(t_0, T, K, q_0, p_0, u_0, state_dim, Gamma, t_terminal)